FUNCION PARA EXTRAER LAS CANCIONES DEL API DE SPOTIFY ⏮️⏹️⏩
📋 Resumen del código actualizado:
✅ Géneros: chill y reggaeton
✅ Período: 2019-2023 (5 años)
✅ Canciones: 200 por año para cada género
✅ Mercado: España (ES)
✅ CSV separados: Un archivo por género

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id='d18941f535f5419ab6a05e8fc0dc9875',
        client_secret='d3ffe57bf5124fcb83f42c2882773c72'
    )
)

def get_tracks_by_genre_year(genre, year, target=200, market='ES'):
    """
    Extrae tracks sin audio features (credenciales limitadas)
    """
    tracks_data = []
    offset = 0
    limit = 50  # máximo permitido por Spotify

    while len(tracks_data) < target:
        try:
            results = sp.search(
                q=f'genre:"{genre}" year:{year}',
                type='track',
                limit=limit,
                offset=offset,
                market=market
            )

            items = results['tracks']['items']
            if not items:
                print(f"    No hay más resultados (total: {len(tracks_data)})")
                break

            for track in items:
                if track and track['id']:
                    tracks_data.append({
                        'track_id': track['id'],
                        'name_track': track['name'],
                        'name_artist': track['artists'][0]['name'],
                        'name_album': track['album']['name'],
                        'type': track['type'],
                        'year': year,
                        'duration_ms': track['duration_ms'],
                        'genre': genre,
                        'popularity': track['popularity'],
                        'explicit': track.get('explicit', False),
                        'artist_id': track['artists'][0]['id'] if track['artists'] else None
                    })

                if len(tracks_data) >= target:
                    break

            offset += limit
            time.sleep(0.5)  # pausa para evitar rate limit

        except Exception as e:
            print(f"    Error en offset {offset}: {e}")
            time.sleep(2)
            offset += limit
            if offset > 1000:  # límite de seguridad
                break

    return pd.DataFrame(tracks_data)

# Géneros: chill y reggaeton
genres = ['chill', 'latin']
years = range(2019, 2024)  # 2019 a 2023

genre_dfs = {}

print("="*60)
print("EXTRACCIÓN DE DATOS DE SPOTIFY (SIN AUDIO FEATURES)")
print("="*60)
print("⚠️  Nota: Audio features no disponibles con estas credenciales")
print("="*60)

for genre in genres:
    all_years_df = []

    print(f"\n{'='*60}")
    print(f"Extrayendo género: {genre.upper()}")
    print(f"{'='*60}")

    for year in years:
        print(f"  Año {year}...", end=" ")
        df_year = get_tracks_by_genre_year(genre, year, target=200, market='ES')
        print(f"✓ {len(df_year)} canciones")
        all_years_df.append(df_year)
        time.sleep(1)  # pausa entre años

    df_genre = pd.concat(all_years_df, ignore_index=True)

    # Eliminar duplicados
    initial_count = len(df_genre)
    df_genre.drop_duplicates(
        subset=['name_track', 'name_artist'],
        inplace=True
    )
    duplicates_removed = initial_count - len(df_genre)

    genre_dfs[genre] = df_genre

    # Guardar cada género en un CSV separado
    filename = f'spotify_{genre}_2019-2023.csv'
    df_genre.to_csv(filename, index=False, encoding='utf-8')

    print(f"\n✓ Total de canciones extraídas: {initial_count}")
    print(f"✓ Duplicados eliminados: {duplicates_removed}")
    print(f"✓ Canciones únicas: {len(df_genre)}")
    print(f"✓ Archivo guardado: {filename}")
    
    # Mostrar distribución por año
    print(f"\nDistribución por año:")
    print(df_genre['year'].value_counts().sort_index())
    print("\n")

print("="*60)
print("EXTRACCIÓN COMPLETADA")
print("="*60)
print(f"\nArchivos generados:")
for genre in genres:
    print(f"  📁 spotify_{genre}_2019-2023.csv")
    print(f"     Canciones: {len(genre_dfs[genre])}")

print("\n💡 Para obtener audio features (danceability, energy, tempo, valence):")
print("   Genera nuevas credenciales en https://developer.spotify.com/dashboard")

# Opcional: acceder a los DataFrames
# df_chill = genre_dfs['chill']
# df_reggaeton = genre_dfs['reggaeton']

EXTRACCIÓN DE DATOS DE SPOTIFY (SIN AUDIO FEATURES)
⚠️  Nota: Audio features no disponibles con estas credenciales

Extrayendo género: CHILL
  Año 2019... ✓ 200 canciones
  Año 2020... ✓ 200 canciones
  Año 2021... ✓ 200 canciones
  Año 2022... ✓ 200 canciones
  Año 2023... ✓ 200 canciones

✓ Total de canciones extraídas: 1000
✓ Duplicados eliminados: 338
✓ Canciones únicas: 662
✓ Archivo guardado: spotify_chill_2019-2023.csv

Distribución por año:
year
2019    151
2020    130
2021    144
2022    122
2023    115
Name: count, dtype: int64



Extrayendo género: LATIN
  Año 2019... ✓ 200 canciones
  Año 2020... ✓ 200 canciones
  Año 2021... ✓ 200 canciones
  Año 2022... ✓ 200 canciones
  Año 2023... ✓ 200 canciones

✓ Total de canciones extraídas: 1000
✓ Duplicados eliminados: 68
✓ Canciones únicas: 932
✓ Archivo guardado: spotify_latin_2019-2023.csv

Distribución por año:
year
2019    196
2020    182
2021    188
2022    182
2023    184
Name: count, dtype: int64


EXTRACCIÓN COMPLETADA

A

In [6]:
df_latin = genre_dfs['latin']

In [7]:
df_latin

,track_id,name_track,name_artist,name_album,type,year,duration_ms,genre,popularity,explicit,artist_id
0,703iVQrfbQsXt7Uzgy1h8k,Cristina,Sebastian Yatra,FANTASÍA,track,2019,201626,latin,72,False,07YUOmWljBTXwIseAUd9TW
1,0fea68AdmYNygeTGI4RC18,LA CANCIÓN,J Balvin,OASIS,track,2019,242573,latin,89,False,1vyhD5VmyZ7KMfW5gqLgo5
2,1aUGxXCz9QimNkKeiJtzZ7,105 F Remix,KEVVO,105 F Remix,track,2019,464200,latin,68,True,4QrBoWLm2WNlPdbFhmlaUZ
3,38kLHBjtWJaPXcyFhwRVtI,Kemba Walker,Eladio Carrion,Kemba Walker,track,2019,152960,latin,66,True,5XJDexmWFLWOkjOEjOVX3e
4,0jeS8PyyIRAG0A3t5DtL71,La Playa,Myke Towers,La Playa,track,2019,207731,latin,67,True,7iK8PXO48WeuP03g8YR51W
...,...,...,...,...,...,...,...,...,...,...,...
995,7Ec2XdY7cs14ONZMgsDx4K,DOM1NGO,Milo j,111,track,2023,188742,latin,69,False,19HM5j0ULGSmEoRcrSe5x3
996,1fcOwu14PSWy2JJhpGIOoL,Un Rodeoooo,Rels B,afroLOVA 23',track,2023,136082,latin,65,False,2IMZYfNi21MGqxopj9fWx8
997,2ECSDcH0Fk1yRO5KkVZwKn,UN DESEO,Mora,ESTRELLA,track,2023,218883,latin,67,True,0Q8NcsJwoCbZOHHW63su5S
998,5FThh5HSLORWCw9edEIdW5,Ven Bailalo (Reggaeton Mix),Angel Y Khriz,Ven Bailalo (Reggaeton Mix),track,2023,251733,latin,76,False,7npqy7rgdX2D3yfVyWExKo


In [11]:
df_chill = genre_dfs['chill']

In [13]:
df_chill

,track_id,name_track,name_artist,name_album,release_date,type,year,duration_ms,genre,popularity,explicit,artist_id
0,64KiGzRPDAXwDq8GmTasyD,El Alba - Original Mix,Steen Thottrup,"Chill out Magnifique, Vol. 3",2019-01-03,track,2019,342186,chill,0,False,6tPMvCqpjXjP9Iezu41j3E
1,6oTcFwFYLqTDRhcxPhRh2S,Where We All Soar,Noah Jacob,Where We All Soar,2019-03-07,track,2019,519383,chill,0,False,41AIkbpAuntIq40gd5hIPb
2,572qKBJo8woS7UkmvnWGH9,Meet Me There,Ganga,Mykonos Sunsets (The Chill Out Beach Edition),2019-06-10,track,2019,291213,chill,0,False,0i6vkNic4Oz5lswGJIMsim
3,0lmNI1cIRB3E0TXUTvEfzB,Sensuality,Roberto Sol,"Hotel Lounge, Vol. 6",2019-06-14,track,2019,245250,chill,0,False,55UZtehjnrMmLlIk6UXjxS
4,4A1kyOV7QA8GK1Yzshfw7O,Heaven - Original Mix,Five Seasons,"Lakeside Chill Sounds, Vol. 18",2019-12-13,track,2019,291904,chill,0,False,0vm9SrJaDy3ANXOHU2t6pe
...,...,...,...,...,...,...,...,...,...,...,...,...
992,6f2UfZ5d9c8gabzUgDoi5L,For Your Love,Olga Chung,Where Is The Love,2023-09-01,track,2023,172505,chill,0,False,6UUbraIaaBKQGCZoSOmn0S
993,7hrxTWu4vzxF4uIKlsmk7t,LIMBO - Sped Up Version,keshi,Spedup Songs,2023-10-12,track,2023,186507,chill,0,False,3pc0bOVB5whxmD50W79wwO
995,7F3RkDlbi4H8NsxYPUgjlb,A Primera Hora,Paco Fernandez,Andalucía Chill - Mirador Spa,2023-06-09,track,2023,219961,chill,0,False,0wjmKODfAPpZUNgUfL9VIi
996,6lFTTb1l5SggMYCEYtKERc,It's Hard - Mystic Instrumental Mix,Artenovum,"Café de la Cita, Vol. 7 (Jazzy Bar Lounge & Ch...",2023-07-07,track,2023,247546,chill,0,False,3SPG8VuJfCW9YUtiXItgNP


In [15]:
filename = f'spotify_{genre}_2019-2023.csv'
df_genre.to_csv(filename, index=False, encoding='utf-8')
```

**Esto crea:**
- `spotify_chill_2019-2023.csv` → Todas las canciones de chill
- `spotify_reggaeton_2019-2023.csv` → Todas las canciones de reggaeton

---

### **Parámetros importantes de `to_csv()`:**

| Parámetro | Valor | ¿Para qué sirve? |
|-----------|-------|------------------|
| `filename` | `f'spotify_{genre}_2019-2023.csv'` | Nombre dinámico del archivo |
| `index=False` | False | No guarda el índice como columna |
| `encoding='utf-8'` | UTF-8 | Permite caracteres especiales (ñ, á, etc.) |

---

### **Paso 5: Se repite el proceso para el siguiente género**

El bucle vuelve a empezar con el siguiente género y crea su propio CSV.

---

## 🎯 Resultado final:

Cuando el código termine de ejecutarse, tendrás en tu carpeta:
```
📁 tu_carpeta/
  📄 spotify_chill_2019-2023.csv      (~800-1000 canciones)
  📄 spotify_reggaeton_2019-2023.csv  (~800-1000 canciones)

SyntaxError: invalid decimal literal (2303172012.py, line 6)